# 1. PBDOT calculator

This module determines different physical effects on the orbital period derivative.
There are two versions of the Galatic model.

By default, it uses the same model in Jang et al. 2023 in prep to A&A.
One can use McMillan et al. 2017 model.

Check the part "def pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations):"

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import numpy.random as nr
from astropy import units as u
from astropy.constants import G
from astropy.constants import c
from astropy import constants as const
from astropy.coordinates import SkyCoord
import sys, math, numpy as N, matplotlib.pyplot as P
from libstempo.libstempo import *
import libstempo
import libstempo.plot as LP, libstempo.toasim as LT
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import libstempo as T
from astropy.stats import LombScargle
import subprocess


class params:
    
    def __init__(self):
        pass
    
    def call_parameters(name):
        call=(open(name).read())
        call=call.split('\n')

        param_array=np.zeros(0)
        for i in range(0,len(call)):
            each_line=np.array(call[i].split(' ')) 
            each_line=each_line[each_line!='']
            if each_line[0]=='CLK':
                break
            else:
                pass
            param_array=np.append(param_array,each_line)
        return param_array
    
    def param_finder(array,param):
        param_idx=np.where(array==param)
        param_name=array[param_idx]
        param_val=array[param_idx[0]+1]
        param_err=array[param_idx[0]+3]
        if param=='RAJ' or param=='DECJ': 
            if param=='RAJ':
                head='h'
                head_sub='d'
                param_idx_sub=np.where(array=='DECJ')
                param_name_sub=array[param_idx_sub]
                param_val_sub=array[param_idx_sub[0]+1]
                param_err_sub=array[param_idx_sub[0]+3]
            if param=='DECJ':
                head='d'
                head_sub='h'
                param_idx_sub=np.where(array=='RAJ')
                param_name_sub=array[param_idx_sub]
                param_val_sub=array[param_idx_sub[0]+1]
                param_err_sub=array[param_idx_sub[0]+3]

            coord=''
            err_part=''
            locator=0

            for i in param_val[0]:
                if i==':': 
                    if locator==0:
                        i=head

                    locator=locator+1
                if locator<=1:
                    coord=coord+i
                if locator>1:
                    err_part=err_part+i

            upper=float(err_part[1:])+float(param_err[0])
            lower=float(err_part[1:])-float(param_err[0])

            coord_upper=coord+'m'+str(upper)
            coord_lower=coord+'m'+str(lower)

            coord_sub=''
            err_part_sub=''
            locator_sub=0
            
            for i in param_val_sub[0]:
                if i==':': 
                    if locator_sub==0:
                        i=head_sub
                    locator_sub=locator_sub+1
                if locator_sub<=1:
                    coord_sub=coord_sub+i
                if locator_sub>1:
                    err_part_sub=err_part_sub+i


            sub_part=float(err_part_sub[1:])
            coord_sub=coord_sub+'m'+str(sub_part)
             

            if param=='RAJ':
                coord_lw= SkyCoord(coord_upper, coord_sub, frame='icrs')
                coord_up= SkyCoord(coord_lower, coord_sub, frame='icrs')  
                mean=(coord_lw.galactic.frame.l.deg+coord_up.galactic.frame.l.deg)/2
                err=(mean-coord_lw.galactic.frame.l.deg)

            if param=='DECJ':
                coord_lw= SkyCoord(coord_sub, coord_upper, frame='icrs')
                coord_up= SkyCoord(coord_sub, coord_lower, frame='icrs')            
                mean=(coord_lw.galactic.frame.b.deg+coord_up.galactic.frame.b.deg)/2
                err=(mean-coord_lw.galactic.frame.b.deg)

            param_val=mean
            param_err=err
                
        else:
            param_val=float(param_val[0])
            param_err=float(param_err[0])
      
        return param_name,param_val,param_err

class PK(params):
    def __init__(self):
        pass
    
    def pbdot(mass_m2, mass_m1, d, dd, param,iterations,model,int_only=False):
        pb=params.param_finder(array_al,'PB')[1]
        dpb=params.param_finder(array_al,'PB')[2]
        pmra=params.param_finder(array_al,'PMRA')[1]
        dpmra=params.param_finder(array_al,'PMRA')[2]
        pmdec=params.param_finder(array_al,'PMDEC')[1]
        dpmdec=params.param_finder(array_al,'PMDEC')[2]
        l=params.param_finder(array_al,'RAJ')[1]
        dl=abs(params.param_finder(array_al,'RAJ')[2])
        b=params.param_finder(array_al,'DECJ')[1]
        db=abs(params.param_finder(array_al,'DECJ')[2])
        if model=='ELL1' or model=='ELL1H':
            eps1=params.param_finder(array_al,'EPS1')[1]
            eps2=params.param_finder(array_al,'EPS2')[1]
            deps1=params.param_finder(array_al,'EPS1')[2]
            deps2=params.param_finder(array_al,'EPS2')[2]
            e1=np.random.normal(eps1,deps1,iterations)
            e2=np.random.normal(eps2,deps2,iterations)
            e=np.sqrt(e1**2+e2**2)
        else:
            e=params.param_finder(array_al,'ECC')[1]
            de=params.param_finder(array_al,'ECC')[2]

        def peak_finder(data):
                '''Returns the peak location in the resulting mcmc distribution.'''

                meas=np.percentile(data,[50,50-34.1,50+34.1])
                cut_off=np.percentile(data,[5])

                return meas[0],meas[1],meas[2],meas[1]-meas[0],meas[2]-meas[0],cut_off 


        
        def pbd_gw(mass_m2, mass_m1, e,pb):
            t0=4.925490947*1e-6
            pb=np.random.normal(pb,dpb,iterations)*u.day.to('second')
            m=mass_m2+mass_m1
            return -192*np.pi/5*(pb/2/np.pi)**(-5/3)*(1+73/24*e**2+37/96*e**4)*(1-e**2)**(-7/2)*t0**(5/3)*mass_m2*mass_m1*m**(-1/3)


        def pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations):

            l=np.random.normal(l,dl,iterations)#np.pi/180 #rad
            b=np.random.normal(b,db,iterations) #rad
            

            
            l=l*u.deg.to('radian')
            b=b*u.deg.to('radian')
            pb=np.random.normal(pb,dpb,iterations)*u.day.to('second') #s
            
            d=np.random.normal(d,dd,iterations)*u.pc.to('meter') #m
            ro=np.random.normal(8275,34,iterations)*u.pc.to('meter') #m
            theta=np.random.normal(240.5*u.km.to('m'),4.1*u.km.to('m'),iterations) #m/s

            beta=d/ro*np.cos(b)-np.cos(l)
            z=abs(d*np.sin(b))/1000/u.pc.to('meter') #kpc
            k=(2.27*z+3.68*(1-np.exp(-4.31*z)))*1e-11 #m/s2
            k=np.random.normal(k,abs(k*0.1),iterations)
            c= const.c.value #m/s

            pbd_gal=-k*abs(np.sin(b))/c-theta**2/ro/c*(np.cos(l)+beta/(beta**2+np.sin(l)**2))*np.cos(b)
            gal=(pbd_gal*pb)
            return gal
        
        
        # If you want to use McMillan et al. (2017) galaxy model, use the code below:
        '''
        def pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations):

            l=np.random.normal(l,dl,iterations)#np.pi/180 #rad
            b=np.random.normal(b,db,iterations) #rad
            

            
            l=l*u.deg.to('radian')
            b=b*u.deg.to('radian')
            pb=np.random.normal(pb,dpb,iterations)*u.day.to('second') #s
            
            d=np.random.normal(d,dd,iterations)*u.pc.to('meter') #m
            ro=np.random.normal(8275,34,iterations)*u.pc.to('meter') #m
            theta=np.random.normal(240.5*u.km.to('m'),4.1*u.km.to('m'),iterations) #m/s

            beta=d/ro*np.cos(b)-np.cos(l)
            z=abs(d*np.sin(b))/1000/u.pc.to('meter') #kpc
            k=(2.27*z+3.68*(1-np.exp(-4.31*z)))*1e-11 #m/s2
            k=np.random.normal(k,abs(k*0.1),iterations)
            c= const.c.value #m/s

            pbd_gal=-k*abs(np.sin(b))/c-theta**2/ro/c*(np.cos(l)+beta/(beta**2+np.sin(l)**2))*np.cos(b)
            gal=(pbd_gal*pb)
            return gal        
        '''

        def pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations):    


            ua=np.random.normal(pmra,dpmra,iterations)*4.8481368110954e-9 #mas/yr to rad/yr
            ud=np.random.normal(pmdec,dpmdec,iterations)*4.8481368110954e-9
            u=ua**2+ud**2
            d= np.random.normal(d,dd,iterations)*3.09e+16 # to m
            c=const.c.value/3.17098e-8 # to m/yt
            pb=np.random.normal(pb,dpb,iterations)*0.00273973 # to yr
            u=(ua**2+ud**2)
            shk=u*pb*d/c
            return shk
        
        # intrinsic pbdot based on the obtained values
        gal=np.mean(pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations))
        shk=np.mean(pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations))
        
        gal_dist=pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations)
        shk_dist=pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations)

        gal=peak_finder(pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations))[0]
        gal_err_up=peak_finder(pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations))[4]
        gal_err_low=peak_finder(pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations))[3]
        
        
        
        shk=peak_finder(pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations))[0]
        shk_err_up=peak_finder(pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations))[4]
        shk_err_low=peak_finder(pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations))[3]
        #mass-mass calculation
        if int_only==False:
            pb_gw_dist=pbd_gw(mass_m2, mass_m1,e,pb)
            pb_gw=peak_finder(pb_gw_dist)[0]
            pb_gw_up=peak_finder(pb_gw_dist)[4]
            pb_gw_low=peak_finder(pb_gw_dist)[3]

            print('gal:',gal, u"   \u00B1" ,gal_err_up,gal_err_low)
            print('shk:',shk, u"   \u00B1" ,shk_err_up, shk_err_low)
            print('gw:',pb_gw, "    +" ,pb_gw_up, ' ' ,pb_gw_low)
            print('--------------------------------')
            
            
            total=gal_dist+shk_dist+pb_gw_dist
            
            total_val=peak_finder(total)[0]
            total_err_up=peak_finder(total)[4]
            total_err_low=peak_finder(total)[3]
            
            #ext_err=np.sqrt(gal_err**2+shk_err**2)
            #print('ext:', gal+shk, "    \u00B1" ,ext_err)
            print('total:', total_val, "    +" ,total_err_up, "   -" ,total_err_low)
            print('      ')
        #if int_only==True:

         #   print('gal:',gal, u"   \u00B1" ,gal_err)
         #   print('shk:',shk, u"   \u00B1" ,shk_err)
          #  print('--------------------------------')
          #  ext_err=np.sqrt(gal_err**2+shk_err**2)
          #  print('ext:', gal+shk, "    \u00B1" ,ext_err)
         #   print('      ')
        #return gal,gal_err,shk,shk_err,pb_gw,pb_gw_err
        
    
# Parameters
def mass_function(params):
    A1=params.param_finder(array_al,'A1')[1]
    A1_err=params.param_finder(array_al,'A1')[2]    
    pb=params.param_finder(array_al,'PB')[1]
    pb_err=params.param_finder(array_al,'PB')[2]
    f=(4*np.pi**2/G*(np.around(A1,12)*c.value)**3/(np.around(pb,12)*u.d.to('second'))**2/u.Msun.to('kg')).value
    f_err=(4*np.pi**2/G*c.value**3/u.d.to('second')**2/u.Msun.to('kg')*np.sqrt(
        (3*A1**2*A1_err/pb**2)**2+
        (2*pb_err*A1**3/pb**3)**2)).value
    return f, f_err


# 1. call directory
directory="/home/jiwoong/Desktop/Research/J1439-5501/libstempo/j1439-5501.par"
array_al=params.call_parameters(directory)


# 2. provide the measured mass and mass function.
#If you do not want to calculate PBDOT by GWB, theoretically, omit this part and use 
#PK.pbdot(.....,subtract_only=False) in the next part.
int_only=False
if int_only==False:
    mpdf=pd.read_csv('/home/jiwoong/Desktop/Research/J1439-5501/libstempo/mass_pdf.csv')
    mass_m1=mpdf['Mp']
    mass_m2=mpdf['Mc']
    f= mass_function(params)[0] #around 0.227596605870
    df=mass_function(params)[1] #around 0.000000027349
    binary_model='ELL1H'
if int_only==True:
    pass


# 3. iteration of the distance
d=np.linspace(655,1200,6)
error=0.2 #30% of error assumed
iterations=len(mass_m1)
for distance in d:
    print('d:', distance)
    dd=error*distance
    
    pbd=PK.pbdot(mass_m2,  mass_m1,  distance, dd, params, iterations, binary_model,int_only)


# 2. Forecasting

Using parameter files, including and excluding a target parameter, the code generates mock ToAs.
The likelihood function for two models were compared and this is used as a criteria for the forecasting.

In [ ]:

class Model:

    def __init__(self):
        pass
    
    
    
    def AIC(new,toa_err,mask):

        residual=new[mask]
        toaerr=toa_err[mask]*1e-6
        n=len(toaerr)
        k=18
        
        likeli=1/np.sqrt(2*np.pi*toaerr**2)*np.exp(-residual**2/(2*toaerr**2))
        log_lik=1
        for l in likeli:
            log_lik=log_lik*l
        lnL=np.log(log_lik)
        
        return -lnL

    def BIC(new,toa_err,mask):

        residual=new[mask]
        toaerr=toa_err[mask]*1e-6
        n=len(toaerr)
        k=18

        likeli=1/np.sqrt(2*np.pi*toaerr**2)*np.exp(-residual**2/(2*toaerr**2))
        log_lik=1
        for l in likeli:
            log_lik=log_lik*l
        lnL=np.log(log_lik)
     
        return -lnL

    def red_chisq(origina, new, toa_err, mask):

        residual=np.append(original['res'],new[mask])
        toaerr=np.append(original['err']*1e-6,toa_err[mask]*1e-6)
        chisq=sum(residual**2/toaerr**2)
        return chisq

def add_colorbar_outside(im,ax):
    fig = ax.get_figure()
    bbox = ax.get_position() #bbox contains the [x0 (left), y0 (bottom), x1 (right), y1 (top)] of the axis.
    width = 0.05
    eps = 0.01 #margin between plot and colorbar
    # [left most position, bottom position, width, height] of color bar.
    cax = fig.add_axes([bbox.x1 + eps, bbox.y0, width, bbox.height])
    cbar = fig.colorbar(im, cax=cax)


class toa:

    def __init__(self):
        pass    

    def mjd(data):
        mjd_scat=data.satDay()+data.satSec()
        return mjd_scat    

    def res(data):
        res=data.residuals()
        return res   

    def err(data):
        toaerr=data.toaerrs
        return toaerr*1e-6 


# generate simulation
start_mjd=60460 #60278 #59818
end_mjd=69160.7
phase_ska=61525#61136#60676
days=90

factor=1e-16

# number of realizations
pbdot_wo=0*factor #3.3087294478523456e-15#1.5001824285520343e-14#1.3901382742019989e-14
num_sim=100
# select when will be the detection. hint: aic/err should be bigger than this
sel=3

# set dimmer limit 
set_err_val=6
ef_cntrl=1.1
obs_random_interval=0.3

#location of index of mjd where you want to inspect
index_loc=np.array([100,300,535,545,555,565,575,585,595,605,615,625,635])



start=start_mjd

# call files and make observed arrays

original=pd.read_csv('original.csv')

res=original['res']
toas=original['toa']
err=original['err']

pks=(toas<58589)

toas_pks=np.array(toas)[pks]
err_pks=np.array(err)[pks]
res_pks=np.array(res)[pks]

toas_mk=toas[(toas>57500)&(toas<59719) & (err<set_err_val)]
toas_mk_dim=toas[(toas>57500)&(toas<59719) & (err>set_err_val)]
err_mk=err[(toas>57500)&(toas<59719) & (err<set_err_val)]
err_mk_dim=err[(toas>57500)&(toas<59719) & (err>set_err_val)]
res_mk=res[(toas>57500)&(toas<59719) & (err<set_err_val)]
res_mk_dim=res[(toas>57500)&(toas<59719) & (err>set_err_val)]

# put rms from tempo2. 
mk_rms=4.660#np.mean(err[(toas>57500)&(toas<59719)])
pk_rms=6.366


AIC_appender=np.zeros(0)
BIC_appender=np.zeros(0)

# generate fake observation for simulation.
obs_days=np.arange(start,end_mjd,days)
rand_obs=obs_random_interval*days*np.random.randn(len(obs_days))
obs_day=obs_days+rand_obs
mjd_range=np.append(toas,obs_day[obs_day>59719])

pks=(mjd_range<58589)
mk=(mjd_range>57500)&(mjd_range<59719)

phase0=(mjd_range>=59830) & (mjd_range<start)
phase1=(mjd_range>=start) & (mjd_range<phase_ska)
phase2=(mjd_range>=phase_ska)

phase0_rms=mk_rms
phase1_rms=mk_rms*0.72
phase2_rms=mk_rms*0.28
err_set=np.concatenate((np.array(err),np.array([phase0_rms]*len(phase0[phase0==True])),np.array([phase1_rms]*len(phase1[phase1==True])),
                               np.array([phase2_rms]*len(phase2[phase2==True]))))


#generate realizations

res_fit_val=np.zeros(0)
res_fit_err=np.zeros(0)

mjd_fit_loc=np.zeros(0)

for i in range(0,num_sim):
    print('iteration:',i)
    #generate fake datasets for two cases

    #rand_obs=0.3*days*np.random.randn(len(obs_days))
    sim = LT.fakepulsar(parfile='sim_wt_pbd.par',
                            obstimes=mjd_range,  
                            toaerr=err_set)

    LT.add_efac(sim,efac=ef_cntrl)
    wt_res=sim.residuals()
    
    sim_wo = LT.fakepulsar(parfile='sim_wt_pbd.par',
                            obstimes=mjd_range,  
                            toaerr=err_set)

    sim_wo['PBDOT'].val=pbdot_wo
    LT.add_efac(sim_wo,efac=ef_cntrl)
    wo_res=sim_wo.residuals()
    


# 545 is the maximum length of the tim file

    sim_wo.savetim('iteration.tim')    
        
    BIC=[]
    AIC=[]

    #calculate ICs at each satDay.
        

    for mjd in mjd_range:
        if mjd==mjd_range[0]:
            pass
        else:
            model_wo_bic=  Model.BIC(wo_res,err_set,(mjd_range<mjd))#,'wo_pbdot')  
            model_wt_bic=  Model.BIC(wt_res,err_set,(mjd_range<mjd))#,'wt_pbdot')
            model_wo_aic=  Model.AIC(wo_res,err_set,(mjd_range<mjd))#,'wo_pbdot')  
            model_wt_aic=  Model.AIC(wt_res,err_set,(mjd_range<mjd))#,'wt_pbdot')

            BIC=BIC+[model_wo_bic-model_wt_bic]
            AIC=AIC+[model_wo_aic-model_wt_aic]
            
    for fit_point in index_loc:
        data=pd.read_csv('iteration.tim',skiprows=[i for i in range(fit_point,646)])
        pd.DataFrame(data).to_csv('iteration_sub.tim', index=None)
        sim_wo_ft=T.tempopulsar('sim_wt_pbd.par','iteration_sub.tim')

        #sim_wo_ft['RAJ'].fit = True
        #sim_wo_ft['DECJ'].fit = True
        #sim_wo_ft['F0'].fit = True
        #sim_wo_ft['F1'].fit = True
        #sim_wo_ft['DM'].fit = True
        #sim_wo_ft['DM1'].fit = True
        #sim_wo_ft['TASC'].fit = True
        sim_wo_ft['PB'].fit = True
        #sim_wo_ft['H3'].fit = True
        #sim_wo_ft['STIG'].fit = True
        #sim_wo_ft['A1'].fit = True
        #sim_wo_ft['PMRA'].fit = True
        #sim_wo_ft['PMDEC'].fit = True
        #sim_wo_ft['EPS1'].fit = True
        #sim_wo_ft['EPS2'].fit = True
        sim_wo_ft['PBDOT'].fit = True
        #sim_wo_ft['PB'].fit = True

        for i in range(0,2):
            ret = sim_wo_ft.fit()
        res_fit_val=np.append(res_fit_val,sim_wo_ft['PBDOT'].val)
        res_fit_err=np.append(res_fit_err,sim_wo_ft['PBDOT'].err)
    #append ICs at each realization

    AIC_appender=np.append(AIC_appender,np.array(AIC))
    BIC_appender=np.append(BIC_appender,np.array(BIC))

        
    
mjd_range_chisq=mjd_range[1:] 
num_mjd=len(mjd_range_chisq)       

# calculate mean, std of ICs at each mjd.
AIC_mean=AIC_appender.reshape(num_sim,num_mjd).mean(axis=0)
AIC_std=AIC_appender.reshape(num_sim,num_mjd).std(axis=0)
BIC_mean=BIC_appender.reshape(num_sim,num_mjd).mean(axis=0)
BIC_std=BIC_appender.reshape(num_sim,num_mjd).std(axis=0)


somecondition = True

matplotlib.rc('axes',edgecolor='k')
plt.figure(4) #create one of the figures that must appear with the chart
gs = gridspec.GridSpec(4,1)


min_AIC_range=AIC_mean-AIC_std
max_AIC_range=AIC_mean+AIC_std
min_BIC_range=BIC_mean-BIC_std
max_BIC_range=BIC_mean+BIC_std


# 3. Plotting the figure

Below, one can plot the trends of the ToAs between the perfect model and the weak model.

In [ ]:
f, (resid_fig, likeli_fig, meas_fig) = plt.subplots(3, 1, gridspec_kw={'height_ratios': [20, 10, 10]})

resid_fig.figure.set_size_inches(16, 10)

resid_fig.errorbar(mjd_range,wt_res*1e+6,err_set,fmt='.',label=r'M(D|$\dot{P}_{\rm b}\in$P)')
resid_fig.errorbar(mjd_range,wo_res*1e+6,err_set,fmt='.',label=r'M(D|$\dot{P}_{\rm b}\notin$P)',zorder=100)

resid_fig.axvline(start_mjd,ymin=-60,ymax=60,linestyle=':',color='y',zorder=-100)
resid_fig.axvline(phase_ska,ymin=-60,ymax=60,linestyle=':',color='y',zorder=-100)
resid_fig.text(phase_ska+240,110, 'C',fontsize=15)
resid_fig.text(start_mjd+240,110, 'B',fontsize=15,zorder=100000000)
resid_fig.text(start_mjd-600,110, 'A',fontsize=15)

resid_fig.set_xticks([])
resid_fig.set_ylabel('Residuals ($\mu$s)',fontsize=15, color='k')
resid_fig.set_xlim(min(original['toa'])-100,max(mjd_range)+100)
resid_fig.tick_params(axis='y',  labelsize=15)
resid_fig.legend(fontsize=15)
resid_fig.set_rasterized(True)


likeli_fig.figure.set_size_inches(8, 10)
likeli_fig.axhline(1,xmin=0,xmax=69000,linestyle=':',color='g') 
sel_model=min(mjd_range_chisq[(BIC_mean/BIC_std)>sel])
likeli_fig.axhline(min(BIC_mean[mjd_range_chisq>sel_model]),xmin=0,xmax=69000,linestyle=':',color='r')
likeli_fig.set_yscale('log')
likeli_fig.fill_between(mjd_range_chisq, min_AIC_range, max_AIC_range,
                 facecolor="green", alpha=0.2)         
likeli_fig.plot(mjd_range_chisq,AIC_mean,label=r'$\ln{L}$',color='green') 
likeli_fig.axvline(start_mjd,ymin=-60,ymax=60,linestyle=':',color='y',zorder=-100)
likeli_fig.axvline(phase_ska,ymin=-60,ymax=60,linestyle=':',color='y',zorder=-100)

Z_mjd=mjd_range[1:][index_loc]
mask= [np.around(i,3) in np.around(Z_mjd,3) for i in np.around(mjd_range[1:],3)]

pbd_measure=res_fit_val.reshape(num_sim,len(index_loc)).mean(axis=0)
pbd_unc=res_fit_err.reshape(num_sim,len(index_loc)).mean(axis=0)
    #pbd_measure_err=res_fit_val.reshape(num_sim,len(index_loc)).std(axis=0)
    #pbd_unc_err=res_fit_err.reshape(num_sim,len(index_loc)).std(axis=0)

mask= [i in Z_mjd for i in mjd_range[1:]]
AIC_mean_unc=AIC_mean[mask]
min_AIC_cb=min_AIC_range[mask]
max_AIC_cb=max_AIC_range[mask]
min_BIC_cb=min_BIC_range[mask]
max_BIC_cb=max_BIC_range[mask]
    
    
Z = abs(pbd_measure/pbd_unc)
    #normalize = colors.Normalize(vmin=Z.min(), vmax=Z.max())
    #Z_err=Z*np.sqrt((pbd_measure_err/pbd_measure)**2+(pbd_unc_err/pbd_unc)**2)

color=likeli_fig.scatter(Z_mjd,AIC_mean_unc,edgecolors='none',s=50,c=Z, cmap='RdYlGn')
    
likeli_fig.axhline(min(AIC_mean_unc[Z>3]),xmin=0,xmax=69000,linestyle=':',color='r')
    #ax.axhline(min(Z[Z>3]),xmin=0,xmax=69000,linestyle=':',color='r')

likeli_fig.set_ylabel(r'M(D|$\dot{P}_{\rm b}\notin$P)-M(D|$\dot{P}_{\rm b}\in$P)',fontsize=15, color='k')
likeli_fig.set_xlim(min(original['toa'])-100,max(mjd_range)+100)
likeli_fig.legend(fontsize=15)
likeli_fig.tick_params(axis='both', labelsize=15)
likeli_fig.set_xticks([])

add_colorbar_outside(color,likeli_fig)

meas_fig.figure.set_size_inches(8, 10)
meas_fig.errorbar(Z_mjd,pbd_measure,pbd_unc,marker='.', linestyle='',color='k')
#meas_fig.axhline(6.6243935278705604e-15)
meas_fig.axhline(1.7417306822136477e-15,linestyle=':',color='b')
meas_fig.axhline(1.1530558681173629e-14,linestyle=':',color='b')
meas_fig.set_ylabel(r'$\dot{P}_{\rm b}$',fontsize=15, color='k')
meas_fig.set_xlabel('MJD',fontsize=15, color='k')
meas_fig.set_xlim(min(mjd_range_chisq),max(mjd_range_chisq))
meas_fig.set_ylim(-5e-14,6e-14)
meas_fig.tick_params(axis='both', labelsize=15)
meas_fig.tick_params(axis='x', labelsize=15,rotation=45)    
    
                
                
#plt.colorbar(color, format='%d', orientation='horizontal',shrink=0.6)
plt.savefig('forecast.pdf', bbox_inches='tight', dpi=120)    
plt.show()


